In [1]:
# this is MuZero!
# https://arxiv.org/abs/1911.08265
# It works, but only for CartPole
# Would be happy if it worked for Follower, LunarLander, or Acrobot
# Also, while it's written, it's not using mcts.
# Replace naive_search(which tries n**K) with mcts_search to use

In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline
import tensorflow as tf
import numpy as np
import gym
from tqdm import tqdm, trange
import os,sys
sys.path.append(os.getcwd())

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Make Follower work! Will give interview to anyone who does.
from muzero.follower import Follower
#env = Follower()
env = gym.make("CartPole-v0")
#env = gym.make("MountainCar-v0")
#env = gym.make("LunarLander-v2")
#env = gym.make("Acrobot-v1")

from muzero.model import MuModel
m = MuModel(env.observation_space.shape, env.action_space.n, s_dim=128, K=3, lr=0.001)
print(env.observation_space.shape, env.action_space.n)

from muzero.game import Game, ReplayBuffer
from muzero.mcts import naive_search, mcts_search
replay_buffer = ReplayBuffer(50, 128, m.K)
rews = []

(4,) 2


In [4]:
import random
def play_game(env, m):
    game = Game(env, discount=0.997)
    while not game.terminal():
        game.env.render()
        cc = random.random()
        if cc < 0.05:
            policy = [1/m.a_dim]*m.a_dim
        else:
            policy = naive_search(m, game.observation, T=1)
        game.act_with_policy(policy)
    return game

In [5]:
from muzero.model import reformat_batch
import collections

for j in range(30):
    game = play_game(env, m)
    replay_buffer.save_game(game)
    for i in range(20):
        m.train_on_batch(replay_buffer.sample_batch())
    rew = sum(game.rewards)
    rews.append(rew)
    print(len(game.history), rew, collections.Counter(game.history), m.losses[-1][0])

ImportError: 
    Error occurred while running `from pyglet.gl import *`
    HINT: make sure you have OpenGL install. On Ubuntu, you can run 'apt-get install python-opengl'.
    If you're running on a server, you may need a virtual frame buffer; something like this should work:
    'xvfb-run -s "-screen 0 1400x900x24" python <your_script.py>'
    